**CMP617**



Objetivo: Gerar um classificador de polaridade utilizando word embeddings e LSTM

#Ambiente

In [3]:
# Install environment dependencies
%pip install unidecode nltk pandas numpy matplotlib seaborn scikit-learn yellowbrick setuptools plotly nbformat tensorflow gensim==4.3.1 scipy==1.10.1

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

# Datasets

## AmericanasBR

In [5]:
#baixando os datasets
!curl https://www.inf.ufrgs.br/~viviane/DS/B2W-Reviews01_binario_TEST.csv > B2W-Reviews01_binario_TEST.csv
!curl https://www.inf.ufrgs.br/~viviane/DS/B2W-Reviews01_binario5000_TRAIN.csv > B2W-Reviews01_binario5000_TRAIN.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  981k  100  981k    0     0   705k      0  0:00:01  0:00:01 --:--:--  706k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1657k  100 1657k    0     0   887k      0  0:00:01  0:00:01 --:--:--  887k


In [6]:
df_train = pd.read_csv('B2W-Reviews01_binario5000_TRAIN.csv')
df_test = pd.read_csv('B2W-Reviews01_binario_TEST.csv')

# Classificação com embeddings

In [7]:
from sklearn.model_selection import train_test_split
import random
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.layers import TextVectorization
from keras.layers import Embedding


2025-04-04 14:09:31.338475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 14:09:31.344721: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 14:09:31.355057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743775771.369940   13262 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743775771.374316   13262 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743775771.387207   13262 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [8]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epocas")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

## Pré-processamento

Atividade opcional: verificar resultado com script de pré-processamento do [NILC](http://www.nilc.icmc.usp.br/embeddings).

In [9]:
import re

In [10]:
from pandas.core.common import random_state
#função de pré-processamento
special_chars = "¨'!#$%&()*+,-./:;<=>?@[\]^_`{|}~"
# stop_words = stopwords.words('portuguese')
def preprocess(x):
    new_x = x.replace('"',' ')
    for c in special_chars:
        new_x = new_x.replace(c,' ')
    # new_x = ' '.join([word for word in nltk.word_tokenize(new_x.lower(),language='portuguese') if word not in stop_words]) #removendo stop words
    new_x = re.sub(r'[^\w\s]', ' ', new_x) #removendo pontuação do texto
    new_x = re.sub("http\S+", ' ',new_x) # remove links
    new_x = re.sub("@\w+", ' ',new_x) # remove contas com @
    new_x = re.sub('#\S+', ' ',new_x) # hashtags
    new_x = re.sub('[0-9]+', ' ',new_x) # remove numeros e palavras com numeros
    #new_x = unidecode(new_x) #acentos
    new_x = re.sub("\s+", ' ',new_x) # espaços
    new_x = new_x.strip()
    new_x = new_x.lower()
    return new_x

#pré-processar datasets de treino e teste
df_train['text_original'] = df_train['text']
df_train['text'] = df_train['text'].apply(preprocess)

df_test['text_original'] = df_test['text']
df_test['text'] = df_test['text'].apply(preprocess)

df_train = df_train.sample(n=len(df_train), random_state=42).copy()
df_train.reset_index(drop=True, inplace=True)
len(df_train)

10000

In [11]:
# remove instâncias com texto com comprimento zero ou nulo
df_train = df_train[df_train['text']!='']
df_train = df_train[~df_train['text'].isna()]
df_train.reset_index(drop=True, inplace=True) # reindexa dataframe
len(df_train)

9996

In [12]:
# remove duplicidades
dup = df_train[df_train.duplicated(subset=['text'])]
df_train = df_train.drop(dup.index)
df_train.reset_index(drop=True, inplace=True) # reindexa dataframe
len(df_train)

9926

## Preparando os datasets


In [13]:
len(df_train), len(df_test)

(9926, 6000)

In [14]:
df_train.sample(n=5)

,label,text,label_descr,text_original
6645,1,produto muito bom valor justo supriu a necessi...,positivo,"Produto muito bom. Valor justo, supriu a neces..."
3024,1,gostei da escova muito pratica de usar e chego...,positivo,"Gostei da escova ,muito pratica de usar e cheg..."
5855,1,possui cheiro forte mas o importante é que rea...,positivo,"Possui cheiro forte, mas o importante é que re..."
8968,0,não era o produto q eu esperava pois na compra...,negativo,Não era o produto q eu esperava pois na compra...
5910,0,como posso avaliar um produto que ainda não ch...,negativo,como posso avaliar um produto que ainda não ch...


Separa treino em treino e validação. O conjunto de validação é ser usado durante treinamento da rede.

In [15]:
# split treino em: treino e val
train_samples, val_samples, train_labels, val_labels = train_test_split(
    df_train['text'].values, df_train['label'].values, test_size=0.2, random_state=42) #reprodutibilidade dos datasets gerados

test_samples = df_test['text'].values
test_labels = df_test['label'].values

train_samples.shape, val_samples.shape, test_samples.shape

((7940,), (1986,), (6000,))

In [16]:
train_samples[10], val_samples[10]

('péssimo pedi começo do ano e ainda tá aguardando nota fiscal com previsão de entrega só em abril como uma compradora regular do site estou insatisfeita',
 'lamentavelmente na descrição do produto não informa que precisa utilizar o cabo usb e o vga sendo assim quando comprei o monitor tive que fazer um outro investimento do conversor de vga para hdmi pois meu notebook não tem entrada vga a americanas deveria informar no descritivo do produto essa questão pois o cliente não tem ciência do problema informado acima')

## Criando o codificador para o vocabulário

O `vectorizer` é instanciado usando o `TextVectorization` do Keras que fará a indexação do vocabulario do dataset.

Ou seja, `vectorizer` fará a codificação das instâncias. Aqui o vocabulário considera 20.000 palavras e fará o truncamento ou padding das instâncias para que todas tenham 128 tokens


In [17]:
MAX_LENGHT = 128
vectorizer = TextVectorization(
    max_tokens=20000, #vocabulário maximo
    output_sequence_length=MAX_LENGHT,
    )

2025-04-04 14:09:34.294616: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [18]:
# antes de adapatar vectorizer a um conjunto de textos o cvocabulario so tem 2 tokens:
len(vectorizer.get_vocabulary()), vectorizer.get_vocabulary()

(2, ['', '[UNK]'])

In [19]:
# adapapta:
vectorizer.adapt(train_samples)
len(vectorizer.get_vocabulary())

12471

In [20]:
# agora vemos os tokens do dataset começando pelos mais frequentes:
voc = vectorizer.get_vocabulary()
print(f'Vocabulário tem {len(voc)} tokens. Os primeiros 10 tokens são:')
voc[:100]

Vocabulário tem 12471 tokens. Os primeiros 10 tokens são:


['',
 '[UNK]',
 'o',
 'e',
 'a',
 'de',
 'não',
 'produto',
 'que',
 'muito',
 'é',
 'com',
 'do',
 'um',
 'para',
 'da',
 'em',
 'no',
 'entrega',
 'recomendo',
 'mais',
 'uma',
 'na',
 'bom',
 'mas',
 'chegou',
 'foi',
 'se',
 'comprei',
 'eu',
 'qualidade',
 'as',
 'americanas',
 'prazo',
 'recebi',
 'ainda',
 'bem',
 'me',
 'já',
 'tem',
 'por',
 'como',
 'meu',
 'estou',
 'excelente',
 'até',
 'os',
 'antes',
 'compra',
 'veio',
 'pra',
 'minha',
 'super',
 'sem',
 'só',
 'pois',
 'ótimo',
 'dia',
 'gostei',
 'nao',
 'dias',
 'ser',
 'nem',
 'boa',
 'mesmo',
 'ao',
 'ele',
 'nada',
 'esse',
 'está',
 'comprar',
 'loja',
 'agora',
 'site',
 'ou',
 'tudo',
 'aparelho',
 'pelo',
 'celular',
 'ter',
 'quando',
 'rápida',
 'isso',
 'tive',
 'uso',
 'preço',
 'rápido',
 'entregue',
 'tenho',
 'melhor',
 'problema',
 'funciona',
 'ótima',
 'minhas',
 'das',
 'era',
 'tempo',
 'q',
 'fácil',
 'defeito']

Vetorizando uma sentença:

In [21]:
output = vectorizer([["chuva forte na floresta"]])
output.shape, output

(TensorShape([1, 128]),
 <tf.Tensor: shape=(1, 128), dtype=int64, numpy=
 array([[2267,  691,   22,    1,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0]])>)

In [22]:
# conferindo as palavras que correnspodem a alguns dos índices gerados na condificação:
voc[2267],voc[691], voc[22], voc[1]

('chuva', 'forte', 'na', '[UNK]')

In [23]:
# um dicionario com todas as palavras do vocabulario e seus indices de 0 a len(voc)-1
word_index = dict(zip(voc, range(len(voc))))
len(voc)

12471

In [24]:
word_index['chuva']

2267

## Usando embeddings pré-treinadas
####Há várias opções de embeddings já treinadas em português no site do NILC nesse [link](http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc)

In [25]:
#Opção 1: baixando as embeddings do NILC
# !curl http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s100.zip > glove_s100.zip
# !unzip -o glove_s100.zip

!curl http://143.107.183.175:22980/download.php?file=embeddings/fasttext/skip_s100.zip > skip_s100.zip
!unzip -o skip_s100.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  299M  100  299M    0     0  10.6M      0  0:00:28  0:00:28 --:--:-- 10.6M
Archive:  skip_s100.zip
  inflating: skip_s100.txt           


In [26]:
#embFile = 'fast_cbow_s50.txt'
#embFile = 'fast_skip_s100.txt'
embFile = 'skip_s100.txt'
#embFile = 'w2v_skip_s300.txt'
# embFile = 'glove_s100.txt'
embedding_dim = 100


In [ ]:
#Opção 2 para quem tem as embeddings no seu drive
# from google.colab import drive
# drive.mount('/content/drive')
# embFile = f"/content/drive/MyDrive/Colab Notebooks/embeddings/{embFile}"

: 

In [ ]:
# verificando o arquivo de embeddings e carregando para uma matriz: cada linha da matriz tem o numero de dimensões das embeddings
embeddings_index = {}
with open(embFile) as f:
    l=0
    for line in f:
        if l>0:
            pos = line.find(' ')
            word = line[0:pos]
            vector = line[pos:len(line)]
            vector = vector.replace('\n','')
            vector = vector.strip()
            vector = list(map(float, vector.split(' ')))
            embeddings_index[word] = vector
        l += 1
print(f"Encontrados {len(embeddings_index)} vetores de palavras.")

Verificando palavras do arquivo de embeddings:

In [ ]:
primeira = list(embeddings_index.keys())[0]
print('primeiro token: ',primeira)
print('embeddigns do primeiro token: ',embeddings_index[primeira])
print('---------------')
centesima = list(embeddings_index.keys())[100]
print('centésimo token: ',centesima)
print('embeddigns do centesimo token: ',embeddings_index[centesima])
print('---------------')
ultima = list(embeddings_index.keys())[len(embeddings_index)-1]
print('ultimo token: ',ultima)
print('embeddigns do ultimo token: ',embeddings_index[ultima])


In [ ]:
len(word_index), len(voc),len(embeddings_index)

In [ ]:
word_index['mais']

O proximo passo é efetuar o mapeamento entre as embeddings e as palavras do vocabulário do dataset. Prepara-se uma matriz de embeddings: uma matriz NumPy simples onde a entrada no índice `i` é o vetor pré-treinado para a palavra de índice `i` no vocabulário do nosso `vetorizador`.

In [ ]:
embedding_dim

In [ ]:
num_tokens = len(voc)
hits = 0
misses = 0

# Preparando  a matriz de embeddings
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    if word in embeddings_index:
        # print(word, i, len(embeddings_index[word]))
        # Palavras não encontradas no índice de embeddings terão seus valores na matriz zerados
        embedding_matrix[i] = embeddings_index[word]
        hits += 1
    else:
        misses += 1
print(f"Mapeadas {hits} palavras e {misses} não existem nas embeddings (OOV))")


In [ ]:
# Tamanho da matrix de embeddings e do vocabulario:
embedding_matrix.shape, len(vectorizer.get_vocabulary())

## Preparando o modelo

### Embedding layer

A matriz `embedding_matrix` é usada na definição do layer de embeddings.

In [ ]:
num_tokens, embedding_dim, embedding_matrix.shape

In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    trainable=False, #mantem o layer de embeddgins com os valores que está e nao treina isso junto coma  rede
)
embedding_layer.build((1,))
#usa os pesos das embeddings
embedding_layer.set_weights([embedding_matrix])

Atividade opcional: treinar as embeddgins do dataset (por exemplo usando gensim, ou usando a propria rede).

### LSTM

In [ ]:
import numpy as np
import keras
from keras import layers


In [ ]:
#configura um vetor de inputs que tera os identificadores do vocabulario
inputs = keras.Input(shape=(None,), dtype="int32")

# converte as embeddgins de cada input
embedded_sequences = embedding_layer(inputs)

# adiciona uma LSTM bidirecional
x = layers.Bidirectional(layers.LSTM(64))(embedded_sequences)

# adiciona um classificador binário
outputs = layers.Dense(1, activation="sigmoid")(x)

# Cria o modelo
model = keras.Model(inputs, outputs)

#Exibe o resumo do modelo e suas camadas
model.summary()

Estamos usando `binary_crossentropy` como loss para a classificação binária.

[`Adam`](https://keras.io/api/optimizers/adam/) é o otimizador usado com `learning_rate` default de 0.001.

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
keras.utils.plot_model(model, show_layer_names=False, rankdir='LR', show_layer_activations=True)

## Treinando o modelo

Convertendo o texto dos 3 datasets (treino, val e teste) em vetores com índices do vocabulário, usando o vetorizador definido anteriormente.

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()
x_test = vectorizer(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

x_train.shape, x_val.shape, x_test.shape

In [ ]:
i=10
train_samples[i], x_train[i]

In [ ]:
class_names = df_train.label_descr.unique()
class_names

In [ ]:
%%time
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_val, y_val))

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'loss')
plt.ylim(0, None)
plt.subplot(1, 2, 2)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)


## Avaliando a Qualidade do Modelo


A função `model.evaluate` prevê a saída para a entrada fornecida e, em seguida, calcula a função de métricas especificadas em `model.compile` com base em `y_true` e `y_pred`.

O `model.predict` apenas retorna o y_pred, mas ao final devem chegar no mesmo valor.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
df_test.sample(n=5)

In [ ]:
i=10
train_samples[i], x_train[i]

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose = False)
print(f'Test Loss: {test_loss:.5f}')
print(f'Test Accuracy: {test_acc:.5f}')

In [ ]:
y_prob = model.predict(np.array(x_test))
# convertendo o vetor de probabilidads em um vetor de predições binárias
# y_pred = [int(i > .5) for i in y_prob]
y_pred = [int(i > .5) for i in y_prob.flatten()]
print(f"Acurácia: {accuracy_score(y_test, y_pred):.5f}")
y_pred[0:15]


In [ ]:
#matriz de confusão
print(f"Acurácia: {accuracy_score(y_test, y_pred):.5f}")
print(f"F1-macro: {f1_score(y_test, y_pred, average='macro'):.5f}")
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['negative','positive'])
cm_display.plot()
plt.show()

##Examinando as instâncias mal classificadas

In [ ]:
erros  = list (zip(df_test['text'].values,df_test['text_original'].values,y_test,y_pred)) #criando lista com os erros
erros = [item for item in erros if item[2] != item[3]] #removendo as instâncias corretas
df_erros = pd.DataFrame(erros,columns =['Texto','Original','True','Pred']) #gerando um dataframe para ficar mais fácil de trabalhar

In [ ]:
#acrescentando colunas FP e FN no dataframe com os erros
df_erros['FP'] = df_erros.apply(lambda x: 1 if ((x['Pred']==1) & (x['True']==0)) else 0, axis=1)
df_erros['FN'] = df_erros.apply(lambda x: 1 if ((x['Pred']==0) & (x['True']==1)) else 0, axis=1)

In [ ]:
print('Há ', len(df_erros),' instâncias mal classificadas.')

In [ ]:
#inspecionando os falsos negativos
df_erros[df_erros['FN']==1]

In [ ]:
#inspecionando os falsos positivos
df_erros[df_erros['FP']==1]

# Persistência

Para persistencia do modelo e utilizar o mesmo sem precisar retreinar:

https://www.tensorflow.org/guide/keras/save_and_serialize